# Quantify

In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
import spatialdata as sd

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.object_quantification.controller import QuantificationController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


## Read in config

In [ ]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_analysis_settings(config_path)

# Specifies if a table with a given name should be overwritten if it already exists. 
# If False, the pipeline will throw an error if it tries to overwrite an existing resource.
# If True, the pipeline will overwrite existing resources. Use with caution!
OVERWRITE_FLAG = True

2026-02-09 14:49:56.357 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Define the logger

In [3]:
log_file = config.log_dir_path / f"rois_quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

## Define ROIs for processing

In [4]:
roi_dir = config.analysis_dir / 'rois'
path_list = [roi_dir / f for f in os.listdir(roi_dir)]
path_list.sort()
path_list

[WindowsPath('../examples/output/sample_analysis/rois/ROI_000.zarr'),
 WindowsPath('../examples/output/sample_analysis/rois/ROI_001.zarr')]

In [ ]:
# # create a subset of rois (optional)
# path_list = path_list[:1]

## Setup quantifiers

In [ ]:
# setup quantification controllers
quant_controller_list = [] 
qc_prefix = config.qc.prefix

for quant in config.quant:

    table_name = quant.name
    masks_keys = quant.masks
    mask_to_annotate = quant.layer_connection
    qc_mask = quant.qc_mask

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{mask_to_annotate}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        mask_to_annotate=mask_to_annotate,
        overwrite=OVERWRITE_FLAG,
        add_qc_masks = qc_mask,
        qc_prefix = qc_prefix,
    )

    quant_controller_list.append(controller) 

2026-02-09 15:18:03.284 | INFO     | __main__:<module>:11 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell', 'ring': 'ring', 'cyto': 'cytoplasm'} and connection to 'instanseg_cell' mask


## Run ROIs Quantification

In [6]:
for sd_path in path_list:
    
    # load data
    logger.info(f'Quantifying {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

2026-02-09 15:19:01.441 | INFO     | __main__:<module>:4 - Quantifying ROI_000.zarr
2026-02-09 15:19:02.219 | INFO     | plex_pipe.object_quantification.controller:validate_sdata_as_input:120 - Channels not specified. Quantifying all (3) existing channels: ['CD45', 'DAPI', 'NaKATPase'].
2026-02-09 15:19:02.220 | INFO     | plex_pipe.object_quantification.controller:run:558 - Spatial Data object is valid and ready for quantification.
2026-02-09 15:19:03.411 | INFO     | plex_pipe.object_quantification.controller:build_obs:200 - Quantifying morphology features for mask 'nucleus'
2026-02-09 15:19:23.461 | INFO     | plex_pipe.object_quantification.controller:build_obs:200 - Quantifying morphology features for mask 'cell'
2026-02-09 15:19:52.372 | INFO     | plex_pipe.object_quantification.controller:build_obs:200 - Quantifying morphology features for mask 'ring'
2026-02-09 15:20:18.444 | INFO     | plex_pipe.object_quantification.controller:build_obs:200 - Quantifying morphology features 

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\spatialdata\models\models.py:1183: UserWarning: Converting `region_key: region` to categorical dtype.
  convert_region_column_to_categorical(adata)


2026-02-09 15:21:08.906 | INFO     | plex_pipe.object_quantification.controller:run:564 - Quantification complete. Resulting AnnData has 12451 observations and 25 variables.
2026-02-09 15:21:08.932 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:32 - Table instanseg_table present in the spatialdata object.
2026-02-09 15:21:08.932 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:41 - Centroids: centroid_cell present in the anndata table instanseg_table.
2026-02-09 15:21:09.075 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'label' (key: qc_exclude_label).
2026-02-09 15:21:09.086 | INFO     | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:136 - Applied QC exclusion for marker 'CD45' using shapes 'qc_exclude_CD45'.
2026-02-09 15:21:09.127 | INFO     | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:136 - Applied QC exclusion for marker 'DAPI' 

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env2\Lib\site-packages\spatialdata\models\models.py:1183: UserWarning: Converting `region_key: region` to categorical dtype.
  convert_region_column_to_categorical(adata)


2026-02-09 15:23:09.202 | INFO     | plex_pipe.object_quantification.controller:run:564 - Quantification complete. Resulting AnnData has 12167 observations and 25 variables.
2026-02-09 15:23:09.243 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:32 - Table instanseg_table present in the spatialdata object.
2026-02-09 15:23:09.243 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:41 - Centroids: centroid_cell present in the anndata table instanseg_table.
2026-02-09 15:23:09.373 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'label' (key: qc_exclude_label).
2026-02-09 15:23:09.373 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'CD45' (key: qc_exclude_CD45).
2026-02-09 15:23:09.386 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'DAPI' (key: qc_exc